# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will use the dataset 'Healthcare For All' building a model to predict who will donate (TargetB) and then - how much they will give (TargetD) (will be used for lab on Friday). You will be using `files_for_lab/categorical.csv, numerical.csv, and target.csv` which can be found at this link.
[link to data](https://github.com/ta-data-remote/lab-random-forests/tree/master/files_for_lab)
You will need to download the data locally.  Remember to add the files to your .gitignore.

### Scenario

You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

### Instructions

In this lab, we will take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.  You should fork and clone this Repo and begin a new Jupyter notebook.

Here are the steps to be followed (building a simple model without balancing the data):


**Everyone is starting with the same cleaned data**

 

**Begin the Modeling here**
- Look critically at the dtypes of numerical and categorical columns and make changes where appropriate.
- Concatenate numerical and categorical back together again for your X dataframe.  Designate the TargetB as y.
  - Split the data into a training set and a test set.
  - Split further into train_num and train_cat.  Also test_num and test_cat.
  - Scale the features either by using MinMax Scaler or a Standard Scaler. (train_num, test_num)
  - Encode the categorical features using One-Hot Encoding or Ordinal Encoding.  (train_cat, test_cat)
      - **fit** only on train data, transform both train and test
      - again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
  - Fit a logistic regression (classification) model on the training data.
  - Check the accuracy on the test data.

**Note**: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model has changed.


## Load libraries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

## Read datasets

In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
target = pd.read_csv('target.csv')

In [3]:
display(categorical.shape)
display(numerical.shape)
display(target.shape)

(95412, 22)

(95412, 315)

(95412, 2)

In [4]:
pd.set_option('display.max_columns', None)
numerical

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

In [5]:
display(target.isnull().sum().sum())
display(numerical.isnull().sum().sum())
display(categorical.isnull().sum().sum())

0

0

0

In [6]:
add_to_categorical = []

for i in numerical.columns:
    if numerical[i].nunique()<20:
        add_to_categorical.append(i)
        
display(add_to_categorical)
len(add_to_categorical)

# variables that could be in fact categorical:

['INCOME',
 'WEALTH1',
 'WEALTH2',
 'HV3',
 'HV4',
 'HHD8',
 'TPE7',
 'SEC3',
 'ANC6',
 'ANC13',
 'ANC15',
 'HC15',
 'MHUC2',
 'CARDPM12',
 'HPHONE_D',
 'RFA_2F']

16

In [7]:
# check dtypes
for i in add_to_categorical:
    print(numerical[i].dtype)

int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64


In [8]:
for i in add_to_categorical:
    display(numerical[i].value_counts())
    
# Variables that I will consider categorical:
# INCOME
# WEALTH1
# WEALTH2 
# ANC6 (Percent Hungarian Ancestry) has to be dropped as it is not ethic to keep it 
# ANC13 (Percent Scottish Ancestry) has to be dropped as it is not ethic to keep it
# ANC15 (Percent Ukranian Ancestry) has to be dropped as it is not ethic to keep it 
# MHUC2 (Median Homeowner Cost w/out Mortgage per Month dollars)
# HPHONE_D (Indicator for presence of a published home phone number)
# RFA_2F (Frequency code for RFA_2)

INCOME
5    36737
2    13114
4    12732
1     9022
3     8558
6     7778
7     7471
Name: count, dtype: int64

WEALTH1
9    52317
8     6793
7     6198
6     5825
5     5280
4     4810
3     4237
2     4085
1     3454
0     2413
Name: count, dtype: int64

WEALTH2
9    50346
8     5975
7     5684
6     5497
5     5351
4     5074
3     5046
2     4971
1     4191
0     3277
Name: count, dtype: int64

HV3
3     21368
4     17948
2     17878
5     12466
6      7939
7      5124
1      3173
8      2860
9      2182
12     1297
10     1082
11      999
0       904
13      192
Name: count, dtype: int64

HV4
2     18818
3     18473
4     16117
5     11334
1      9676
6      7374
7      4960
8      3256
9      1945
10     1344
0      1229
11      697
12      144
13       45
Name: count, dtype: int64

...

HC15
0     91426
1      2567
2       864
3       276
4       128
5        67
6        30
7        19
8        10
9         9
30        7
10        4
12        2
11        1
29        1
13        1
Name: count, dtype: int64

MHUC2
2    54924
3    24470
4     7667
1     3722
0     3667
5      962
Name: count, dtype: int64

CARDPM12
6     50685
5     18684
4     16732
3      3821
7      1478
2      1478
8       903
11      479
9       457
10      300
1       279
12       48
13       26
0        21
17        7
14        6
16        4
15        3
19        1
Name: count, dtype: int64

HPHONE_D
1    47765
0    47647
Name: count, dtype: int64

RFA_2F
1    47675
2    20545
3    15291
4    11901
Name: count, dtype: int64

In [9]:
# i checked with my mate which numerical variables from the categorical dataframe are in fact categorical
# i checked from the numericals dataframe which ones are in fact categorical

to_categorical = ['INCOME','WEALTH1', 'WEALTH2', 'MHUC2', 'HPHONE_D', 'RFA_2F', 'DATASRCE', 'DOMAIN_B', 'ODATEW_YR', 'ODATEW_MM', 'DOB_YR', 'DOB_MM', 'MINRDATE_YR', 'MINRDATE_MM', 'MAXRDATE_YR', 'MAXRDATE_MM', 'LASTDATE_YR', 'LASTDATE_MM', 'FIRSTDATE_YR', 'FIRSTDATE_MM']
to_drop = ['ANC6', 'ANC13', 'ANC15']

In [10]:
data = pd.concat((numerical,categorical),axis=1)

In [11]:
data = pd.concat((data, target), axis=1)

In [12]:
# change the dtypes
for i in to_categorical:
    data[i] = data[i].astype(object)

# check dtypes again
for i in to_categorical:
    print(data[i].dtype)

object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object


In [13]:
# drop unethic columns
data = data.drop(to_drop, axis=1)

## Transform columns

In [14]:
# # define a function to clean the column 'HHD8'
# def cleanHHD8(x):
#     if x <= 3:
#         return x
#     elif x > 3:
#         return '4 or more'
#     else:
#         return 'U'

In [15]:
# data['HHD8'] = data['HHD8'].apply(cleanHHD8)

In [16]:
# data['HHD8'].value_counts()

## X-y split

In [17]:
X = data.drop(['TARGET_B', 'TARGET_D'], axis=1)
y = data['TARGET_B']

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### X num-cat split

In [19]:
X_train_num = X_train.select_dtypes(exclude='object')
X_test_num = X_test.select_dtypes(exclude='object')

X_train_cat = X_train.select_dtypes('object')
X_test_cat = X_test.select_dtypes('object')

### Scale

In [20]:
from sklearn.preprocessing import StandardScaler

# scaling standard scaler: make data distributed with mean=0 and std=1
transformer = StandardScaler().fit(X_train_num)
numericals_train_standardized = transformer.transform(X_train_num)
numericals_test_standardized = transformer.transform(X_test_num)

### Encode

In [32]:
# split from the columns to encode and the ones that dont need to be encode
X_train_cat_to_encode = X_train_cat[['STATE', 'HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A']]
X_test_cat_to_encode = X_test_cat[['STATE', 'HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A']]

# drop columns and make new df to concatenate back later
X_train_cat_to_concat = X_train_cat.drop(['STATE', 'HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A'], axis=1).reset_index(drop=True)
X_test_cat_to_concat = X_test_cat.drop(['STATE', 'HOMEOWNR', 'GENDER', 'RFA_2R', 'RFA_2A', 'GEOCODE2', 'DOMAIN_A'], axis=1).reset_index(drop=True)

In [33]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_cat_to_encode)

cols = encoder.get_feature_names_out(input_features=X_train_cat_to_encode.columns)

X_train_encoded = pd.DataFrame(encoder.transform(X_train_cat_to_encode).toarray(),columns=cols)
X_test_encoded = pd.DataFrame(encoder.transform(X_test_cat_to_encode).toarray(),columns=cols)

In [34]:
# concatenate them back
X_train_transformed = pd.concat([X_train_cat_to_concat, X_train_encoded], axis=1)
X_test_transformed = pd.concat([X_test_cat_to_concat, X_test_encoded], axis=1)

In [35]:
X_train_transformed

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,5,9,8,1,0,4,2,1,92,1,48,1,93,1,95,2,95,12,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,6,7,2,2,1,3,3,3,91,1,0,2,91,1,91,6,95,12,91,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,1,9,2,1,1,3,2,95,1,34,1,95,12,94,9,95,12,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,3,0,0,3,1,1,3,3,88,1,31,9,95,2,95,2,95,9,88,10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,9,9,2,0,4,2,2,95,1,54,1,95,6,95,11,96,3,95,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,3,7,4,2,1,2,1,2,90,1,0,2,93,10,95,11,95,11,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
76325,1,1,9,2,1,2,1,3,95,1,0,2,95,1,95,6,95,7,95,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
76326,2,6,9,2,0,3,3,2,95,1,32,1,95,4,96,3,96,3,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
76327,5,9,9,2,1,3,3,2,95,1,0,2,95,1,95,8,95,8,95,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Fit a Logistic Regression model

In [40]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(solver='saga').fit(X_train_transformed, y_train)

C:\Users\pablo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [41]:
# check the performance
display(classification.score(X_train_transformed, y_train))
classification.score(X_test_transformed, y_test)

0.9490888129020425

0.9498506524131426

In [42]:
# check for imbalance
y_train.value_counts()

TARGET_B
0    72443
1     3886
Name: count, dtype: int64

In [43]:
# check confusion matrix
from sklearn.metrics import confusion_matrix
predictions = classification.predict(X_test_transformed)
confusion_matrix(y_test,predictions)

array = confusion_matrix(y_test, predictions)
print('             Predicted Labels')
print('             |   A    |     B')
print('---------------------------------')
print('True label A | ',array[0][0],'|   ', array[0][1])
print('---------------------------------')
print('           B | ',array[1][0],'  |   ', array[1][1])

             Predicted Labels
             |   A    |     B
---------------------------------
True label A |  18126 |    0
---------------------------------
           B |  957   |    0


There is a clear imbalance in the outcome, being 0 the majority class

The model is not predicting correct any donor, and the objective of the model is to predict if a person is going to donate or not. So the rebalance is clearly needed as the model is not useful

I will perform undersampling to rebalance the dataset

### Rebalance dataset 

In [44]:
y_train = y_train.reset_index(drop=True)

In [45]:
trainset = pd.concat([X_train_transformed, y_train], axis=1)

In [46]:
trainset

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,5,9,8,1,0,4,2,1,92,1,48,1,93,1,95,2,95,12,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,6,7,2,2,1,3,3,3,91,1,0,2,91,1,91,6,95,12,91,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5,1,9,2,1,1,3,2,95,1,34,1,95,12,94,9,95,12,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
3,3,0,0,3,1,1,3,3,88,1,31,9,95,2,95,2,95,9,88,10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,3,9,9,2,0,4,2,2,95,1,54,1,95,6,95,11,96,3,95,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,3,7,4,2,1,2,1,2,90,1,0,2,93,10,95,11,95,11,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
76325,1,1,9,2,1,2,1,3,95,1,0,2,95,1,95,6,95,7,95,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
76326,2,6,9,2,0,3,3,2,95,1,32,1,95,4,96,3,96,3,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
76327,5,9,9,2,1,3,3,2,95,1,0,2,95,1,95,8,95,8,95,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [47]:
trainset['TARGET_B'].value_counts()

TARGET_B
0    72443
1     3886
Name: count, dtype: int64

In [48]:
trainset

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,5,9,8,1,0,4,2,1,92,1,48,1,93,1,95,2,95,12,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,6,7,2,2,1,3,3,3,91,1,0,2,91,1,91,6,95,12,91,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5,1,9,2,1,1,3,2,95,1,34,1,95,12,94,9,95,12,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
3,3,0,0,3,1,1,3,3,88,1,31,9,95,2,95,2,95,9,88,10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,3,9,9,2,0,4,2,2,95,1,54,1,95,6,95,11,96,3,95,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,3,7,4,2,1,2,1,2,90,1,0,2,93,10,95,11,95,11,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
76325,1,1,9,2,1,2,1,3,95,1,0,2,95,1,95,6,95,7,95,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
76326,2,6,9,2,0,3,3,2,95,1,32,1,95,4,96,3,96,3,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
76327,5,9,9,2,1,3,3,2,95,1,0,2,95,1,95,8,95,8,95,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [49]:
# separate majority/minority classes
no_donor = trainset[trainset['TARGET_B']==0]
yes_donor = trainset[trainset['TARGET_B']==1]

In [57]:
display(no_donor)
yes_donor

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,5,9,8,1,0,4,2,1,92,1,48,1,93,1,95,2,95,12,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,6,7,2,2,1,3,3,3,91,1,0,2,91,1,91,6,95,12,91,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5,1,9,2,1,1,3,2,95,1,34,1,95,12,94,9,95,12,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
3,3,0,0,3,1,1,3,3,88,1,31,9,95,2,95,2,95,9,88,10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,3,9,9,2,0,4,2,2,95,1,54,1,95,6,95,11,96,3,95,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76324,3,7,4,2,1,2,1,2,90,1,0,2,93,10,95,11,95,11,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
76325,1,1,9,2,1,2,1,3,95,1,0,2,95,1,95,6,95,7,95,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
76326,2,6,9,2,0,3,3,2,95,1,32,1,95,4,96,3,96,3,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
76327,5,9,9,2,1,3,3,2,95,1,0,2,95,1,95,8,95,8,95,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
25,4,9,9,2,0,3,2,2,95,1,62,1,95,6,96,2,96,2,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
26,4,9,9,2,0,1,2,2,96,1,48,1,96,2,96,2,96,2,96,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
65,1,2,4,2,1,4,3,2,90,1,40,9,91,11,95,4,96,3,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
70,5,9,3,1,1,3,3,2,87,1,0,2,89,2,94,12,95,11,87,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
95,4,6,2,3,0,4,1,2,86,1,0,2,94,5,95,6,97,2,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76280,3,2,2,2,1,3,3,3,86,1,61,11,93,4,96,3,96,3,86,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
76281,5,9,8,2,0,1,3,1,86,1,30,1,92,7,95,5,95,9,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
76289,5,9,8,2,0,3,3,1,92,1,0,2,92,2,95,1,96,3,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
76295,2,6,6,3,1,2,3,2,90,1,25,4,95,10,91,8,96,1,90,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


In [50]:
# undersample majority
from sklearn.utils import resample
no_donor_undersampled = resample(no_donor, #<- downsample from here
                                    replace=False, #<- no need to reuse data now, we have an abundance
                                    n_samples = len(yes_donor))

In [51]:
# both sets are the same size
# small, but balanced, and no repeated data
display(yes_donor)
display(no_donor_undersampled)

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
25,4,9,9,2,0,3,2,2,95,1,62,1,95,6,96,2,96,2,94,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
26,4,9,9,2,0,1,2,2,96,1,48,1,96,2,96,2,96,2,96,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
65,1,2,4,2,1,4,3,2,90,1,40,9,91,11,95,4,96,3,90,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
70,5,9,3,1,1,3,3,2,87,1,0,2,89,2,94,12,95,11,87,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
95,4,6,2,3,0,4,1,2,86,1,0,2,94,5,95,6,97,2,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76280,3,2,2,2,1,3,3,3,86,1,61,11,93,4,96,3,96,3,86,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
76281,5,9,8,2,0,1,3,1,86,1,30,1,92,7,95,5,95,9,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
76289,5,9,8,2,0,3,3,1,92,1,0,2,92,2,95,1,96,3,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
76295,2,6,6,3,1,2,3,2,90,1,25,4,95,10,91,8,96,1,90,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
36123,6,7,7,2,0,1,3,1,94,1,49,7,95,7,93,10,95,7,93,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
15643,5,9,8,3,0,1,2,1,89,1,21,10,91,10,96,1,96,12,89,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
16750,5,5,9,2,0,1,3,2,95,1,32,1,95,4,95,4,95,4,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
463,7,9,9,3,0,3,2,2,93,1,56,1,93,1,95,7,96,1,93,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
48148,7,9,9,2,1,2,2,1,86,1,37,11,89,4,95,4,95,8,86,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68898,4,9,9,2,1,1,1,1,95,1,22,1,95,6,95,6,95,6,95,6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
46067,1,2,2,2,1,1,3,2,94,1,35,1,93,9,95,11,95,11,93,9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
37080,4,9,3,2,0,1,2,1,88,1,11,1,89,6,94,4,95,10,89,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
19248,5,9,8,3,0,1,3,1,92,1,16,10,92,5,96,7,96,7,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [60]:
trainset_undersampled = pd.concat([no_donor_undersampled,yes_donor], axis=0)

In [61]:
trainset_undersampled

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
36123,6,7,7,2,0,1,3,1,94,1,49,7,95,7,93,10,95,7,93,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
15643,5,9,8,3,0,1,2,1,89,1,21,10,91,10,96,1,96,12,89,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0
16750,5,5,9,2,0,1,3,2,95,1,32,1,95,4,95,4,95,4,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
463,7,9,9,3,0,3,2,2,93,1,56,1,93,1,95,7,96,1,93,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
48148,7,9,9,2,1,2,2,1,86,1,37,11,89,4,95,4,95,8,86,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76280,3,2,2,2,1,3,3,3,86,1,61,11,93,4,96,3,96,3,86,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
76281,5,9,8,2,0,1,3,1,86,1,30,1,92,7,95,5,95,9,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
76289,5,9,8,2,0,3,3,1,92,1,0,2,92,2,95,1,96,3,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
76295,2,6,6,3,1,2,3,2,90,1,25,4,95,10,91,8,96,1,90,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


### X-y split

In [62]:
y_train_u = trainset_undersampled['TARGET_B']
X_train_u = trainset_undersampled.drop(['TARGET_B'], axis=1)

In [63]:
y_train_u

36123    0
15643    0
16750    0
463      0
48148    0
        ..
76280    1
76281    1
76289    1
76295    1
76303    1
Name: TARGET_B, Length: 7772, dtype: int64

In [64]:
X_train_u

,INCOME,WEALTH1,WEALTH2,MHUC2,HPHONE_D,RFA_2F,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
36123,6,7,7,2,0,1,3,1,94,1,49,7,95,7,93,10,95,7,93,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15643,5,9,8,3,0,1,2,1,89,1,21,10,91,10,96,1,96,12,89,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
16750,5,5,9,2,0,1,3,2,95,1,32,1,95,4,95,4,95,4,95,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
463,7,9,9,3,0,3,2,2,93,1,56,1,93,1,95,7,96,1,93,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48148,7,9,9,2,1,2,2,1,86,1,37,11,89,4,95,4,95,8,86,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76280,3,2,2,2,1,3,3,3,86,1,61,11,93,4,96,3,96,3,86,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
76281,5,9,8,2,0,1,3,1,86,1,30,1,92,7,95,5,95,9,86,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
76289,5,9,8,2,0,3,3,1,92,1,0,2,92,2,95,1,96,3,92,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76295,2,6,6,3,1,2,3,2,90,1,25,4,95,10,91,8,96,1,90,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Fit a model with rebalanced data

In [66]:
classification2 = LogisticRegression(solver='saga').fit(X_train_u, y_train_u)

C:\Users\pablo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [67]:
# check the performance
classification2.score(X_test_transformed, y_test)

0.6035738615521669

In [69]:
# check confusion matrix
from sklearn.metrics import confusion_matrix
predictions2 = classification2.predict(X_test_transformed)
confusion_matrix(y_test,predictions2)

array = confusion_matrix(y_test, predictions2)
print('             Predicted Labels')
print('             |   A    |     B')
print('---------------------------------')
print('True label A | ',array[0][0],'|   ', array[0][1])
print('---------------------------------')
print('           B | ',array[1][0],'  |   ', array[1][1])

             Predicted Labels
             |   A    |     B
---------------------------------
True label A |  11004 |    7122
---------------------------------
           B |  443   |    514


In [71]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
print("precision: ",precision_score(y_test,predictions2))
print("recall: ",recall_score(y_test,predictions2))
print("f1: ",f1_score(y_test,predictions2))

# Recall = True Positive / (True Negative + False Negative) Evaluates how good the model is avoiding to predict Negative when in reality it is Positive.
# Precision = True Positive / (True Negative + False Positive) Evaluates how good the model is avoiding to predict Positive when in reality it is Negative.
# f1 = 2*((Recall * Precision)/(Recall + Precision)) A trade_off between both

precision:  0.06731272917757988
recall:  0.5370950888192267
f1:  0.1196322588153148


After rebalance the dataset, the r2score is worse, but the recall is better


This means that the model is now better predicting donors.